In [4]:
# ============================================================================
# Galesburg Water Quality: Fluoride Concentration Heatmap (Colab Version)
# ============================================================================

# Install required library
!pip install folium -q

import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMap

# ============================================================================
# DATA: Galesburg Water Quality Measurements
# ============================================================================

data = {
    "Name": [
        "Dr. Larry", "Javiour", "Seymour", "SMC", "Hamblin", "Alex", "Gym",
        "Jermaine", "Brandon", "Robyn", "Xander", "Seymour Library", "Longden",
        "Post", "Angela", "Zenitha", "WAC", "Old Main", "GDH", "CFA", "Hope",
        "Alumni Hall", "Karen", "Williams", "Autumn", "Conger Neal", "Kelly",
        "Physics Prof", "Coach Ryan", "Water Plant"
    ],
    "Fluoride_Conc": [
        0.8192880627, 0.8162683227, 0.7802991357, 0.7995213064,
        0.8044206844, 0.7669573964, 0.7847491930, 0.7650737876,
        0.7657220627, 0.6929937851, 0.7730295952, 0.8289044587,
        0.8353531469, 0.8188211073, 0.8490025469, 0.7597411951,
        0.8447338247, 0.7446525647, 0.7776865343, 0.7296687387,
        0.7562845619, 0.7497915981, 0.8434886762, 0.7392882628,
        0.7411841928, 0.7156662174, 0.7632224332, 0.7660066393,
        0.7566631567, 0.8270000000
    ],
    "Distance_m": [
        2769.50, 2499.00, 1145.90, 906.00, 875.50, 4263.00, 1411.00, 2528.00,
        1524.80, 2960.00, 2052.30, 1047.60, 1066.30, 1056.00, 953.10, 2083.70,
        1486.00, 1192.60, 1274.50, 1310.00, 1335.80, 1139.20, 1972.60, 3359.00,
        3623.00, 1099.80, 2889.00, 2675.00, 1105.00, 0.00
    ],
    "Latitude": [
        '40°58\'02"N', '40°56\'17"N', '40°56\'33"N', '40°56\'35"N', '40°56\'38"N',
        '40°59\'04"N', '40°56\'27"N', '40°56\'29"N', '40°57\'33"N', '40°57\'44"N',
        '40°56\'24"N', '40°56\'34"N', '40°56\'28"N', '40°56\'30"N', '40°56\'20"N',
        '40°57\'16"N', '40°56\'35"N', '40°56\'35"N', '40°56\'36"N',
        '40°56\'30"N', '40°56\'37"N', '40°56\'36"N', '40°57\'31"N', '40°57\'17"N',
        '40°56\'39"N', '40°56\'26"N', '40°58\'14"N', '40°57\'27"N', '40°57\'01"N',
        '40°56\'46"N'
    ],
    "Longitude": [
        '90°22\'02"W', '90°21\'25"W', '90°22\'19"W', '90°22\'29"W', '90°22\'29"W',
        '90°23\'03"W', '90°22\'10"W', '90°21\'19"W', '90°22\'45"W', '90°21\'24"W',
        '90°21\'42"W', '90°22\'23"W', '90°22\'26"W', '90°22\'25"W', '90°22\'43"W',
        '90°21\'45"W', '90°22\'03"W', '90°22\'16"W', '90°22\'12"W', '90°22\'13"W',
        '90°22\'09"W', '90°22\'18"W', '90°22\'05"W', '90°20\'47"W', '90°20\'30"W',
        '90°22\'26"W', '90°22\'23"W', '90°21\'24"W', '90°22\'22"W', '90°23\'05"W'
    ],
}

df = pd.DataFrame(data)

# ============================================================================
# FUNCTION: Convert DMS to Decimal Degrees
# ============================================================================

def dms_to_decimal(dms_str: str) -> float:
    """
    Convert coordinates from DMS (Degrees, Minutes, Seconds) to decimal degrees.
    Example: 40°58'02"N -> 40.967222
    """
    direction = dms_str[-1]
    dms_numeric = dms_str[:-1]

    parts = (
        dms_numeric
        .replace("°", " ")
        .replace("'", " ")
        .replace('"', "")
        .split()
    )

    degrees = float(parts[0])
    minutes = float(parts[1]) if len(parts) > 1 else 0.0
    seconds = float(parts[2]) if len(parts) > 2 else 0.0

    decimal = degrees + minutes / 60 + seconds / 3600

    if direction in ["S", "W"]:
        decimal = -decimal

    return decimal

# Convert coordinates to decimal
df["Lat_Decimal"] = df["Latitude"].apply(dms_to_decimal)
df["Lon_Decimal"] = df["Longitude"].apply(dms_to_decimal)

print("✓ Data loaded successfully!")
print(f"Total sampling locations: {len(df)}")
print(
    f"\nFluoride concentration range: "
    f"{df['Fluoride_Conc'].min():.4f} - {df['Fluoride_Conc'].max():.4f} mg/L"
)
print(
    f"Distance from water plant range: "
    f"{df['Distance_m'].min():.1f} - {df['Distance_m'].max():.1f} meters"
)

# ============================================================================
# CREATE INTERACTIVE MAP WITH GREEN GRADIENT HEATMAP
# ============================================================================

# Calculate map center
center_lat = df["Lat_Decimal"].mean()
center_lon = df["Lon_Decimal"].mean()

print(f"\nMap center coordinates: Lat {center_lat:.6f}, Lon {center_lon:.6f}")

# Create base map
map_green = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles="OpenStreetMap",
)

# Prepare heatmap data: [latitude, longitude, weight]
heat_data = [
    [row["Lat_Decimal"], row["Lon_Decimal"], row["Fluoride_Conc"]]
    for _, row in df.iterrows()
]

# Add green gradient heatmap layer
HeatMap(
    heat_data,
    min_opacity=0.3,
    max_opacity=0.85,
    radius=30,
    blur=40,
    gradient={
        0.0: "#c7e9c0",   # Very light green (lowest concentration)
        0.3: "#a1d99b",   # Light green
        0.5: "#74c476",   # Medium green
        0.7: "#41ab5d",   # Green
        0.85: "#238b45",  # Dark green
        1.0: "#005a32",   # Very dark green (highest concentration)
    },
).add_to(map_green)

# Add circle markers for each sampling location
for _, row in df.iterrows():
    conc = row["Fluoride_Conc"]

    # Determine green shade based on concentration
    if conc >= 0.83:
        fill_color = "#005a32"   # Very dark green
    elif conc >= 0.81:
        fill_color = "#238b45"   # Dark green
    elif conc >= 0.78:
        fill_color = "#41ab5d"   # Green
    elif conc >= 0.76:
        fill_color = "#74c476"   # Medium green
    elif conc >= 0.74:
        fill_color = "#a1d99b"   # Light green
    else:
        fill_color = "#c7e9c0"   # Very light green

    folium.CircleMarker(
        location=[row["Lat_Decimal"], row["Lon_Decimal"]],
        radius=9,
        popup=(
            f"<b>{row['Name']}</b><br>"
            f"Fluoride: {conc:.4f} mg/L<br>"
            f"Distance: {row['Distance_m']:.1f} m"
        ),
        color="darkgreen",
        fill=True,
        fillColor=fill_color,
        fillOpacity=0.8,
        weight=2,
    ).add_to(map_green)

# Add Water Plant marker
folium.Marker(
    location=[40.946111, -90.384722],
    popup="<b>Water Plant</b><br>Fluoride: 0.827 mg/L",
    icon=folium.Icon(color="blue", icon="tint", prefix="fa"),
).add_to(map_green)

# Add title overlay
title_html = """
<div style="
    position: fixed;
    top: 10px; left: 50px; width: 600px; height: 60px;
    background-color: white; border: 2px solid grey; z-index: 9999;
    font-size: 16px; font-weight: bold; padding: 10px;
">
    <center>Galesburg Water Quality: Fluoride Concentration Heatmap</center>
</div>
"""
map_green.get_root().html.add_child(folium.Element(title_html))

# Add legend overlay
legend_html = """
<div style="
    position: fixed;
    bottom: 50px; right: 50px; width: 220px; height: 200px;
    background-color: white; border: 2px solid grey; z-index: 9999;
    font-size: 12px; padding: 10px;
">
    <p style="margin: 0; font-weight: bold; text-align: center; padding-bottom: 5px;">
        Fluoride Concentration (mg/L)
    </p>
    <p style="margin: 5px 0;">
        <span style="background-color: #005a32; padding: 3px 10px; color: white;">&nbsp;&nbsp;</span>
        Very High (&gt;0.83)
    </p>
    <p style="margin: 5px 0;">
        <span style="background-color: #238b45; padding: 3px 10px; color: white;">&nbsp;&nbsp;</span>
        High (0.81–0.83)
    </p>
    <p style="margin: 5px 0;">
        <span style="background-color: #41ab5d; padding: 3px 10px; color: white;">&nbsp;&nbsp;</span>
        Medium (0.78–0.81)
    </p>
    <p style="margin: 5px 0;">
        <span style="background-color: #74c476; padding: 3px 10px;">&nbsp;&nbsp;</span>
        Low-Med (0.76–0.78)
    </p>
    <p style="margin: 5px 0;">
        <span style="background-color: #a1d99b; padding: 3px 10px;">&nbsp;&nbsp;</span>
        Low (0.74–0.76)
    </p>
    <p style="margin: 5px 0;">
        <span style="background-color: #c7e9c0; padding: 3px 10px;">&nbsp;&nbsp;</span>
        Very Low (&lt;0.74)
    </p>
</div>
"""
map_green.get_root().html.add_child(folium.Element(legend_html))

# Save the map to HTML (for download or GitHub)
output_path = "galesburg_fluoride_heatmap.html"
map_green.save(output_path)

print("\n✓ Interactive heatmap created successfully!")
print(f"✓ Map saved as: {output_path}")
print(
    "\nColor scheme: Light green (low concentration) → "
    "dark green (high concentration)."
)

# *** DISPLAY MAP INLINE IN COLAB ***
map_green

output_path = "index.html"  # important for GitHub Pages
map_green.save(output_path)
print("Saved:", output_path)


✓ Data loaded successfully!
Total sampling locations: 30

Fluoride concentration range: 0.6930 - 0.8490 mg/L
Distance from water plant range: 0.0 - 4263.0 meters

Map center coordinates: Lat 40.948741, Lon -90.368352

✓ Interactive heatmap created successfully!
✓ Map saved as: galesburg_fluoride_heatmap.html

Color scheme: Light green (low concentration) → dark green (high concentration).
Saved: index.html
